In [3]:
import pandas as pd
import numpy as np
import json

In [4]:
experiment = "experiment_trafficlights"

### Reading input data

In [5]:
with open(f'traffic/input_files/{experiment}/traffic.json', "r") as read_file:
    drivers_input_data = json.load(read_file)
with open(f'traffic/input_files/{experiment}/drivers.json', "r") as read_file:
    traffic_input_data = json.load(read_file)
with open(f'traffic/input_files/{experiment}/lights.json', "r") as read_file:
    lights_input_data = json.load(read_file)

### Reading output data

In [6]:
agent_data = pd.read_csv(f'traffic/output_files/{experiment}/agent_data.csv')
agent_data.head()

,Step,AgentID,X,Y,Velocity,Current_lane,Is_alive
0,1,0,0.415049,166.666667,[0.59356701 0. ],2,True
1,1,1,0.007917,33.333333,[0.40862256 0. ],0,True
2,1,2,0.399195,100.000000,[0.54839379 0. ],1,True
3,1,3,0.000000,100.000000,[0.64069314 0. ],1,False
4,1,4,0.000000,166.666667,[0.33298864 0. ],2,False


## Flow rate

#### Flow rate: The flow rate is the number of cars that pass a certain point per unit of time.

In [7]:
nodes = []
for node in lights_input_data:
    nodes.append(node["pos"])
print("The x positions of nodes:")
print(nodes)

The x positions of nodes:
[0, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200]


In [17]:
t_start = 800
t_end = 1800
measure_point_x = 600
accepted_dist_delta = 10

# A function to calculate flow_rates easy
def calc_flow_rate(t_start,t_end,measure_point_x,accepted_delta):
    return agent_data[(agent_data["Is_alive"]==True)&(agent_data["Step"]>t_start)&(agent_data["Step"]<t_end)&(abs(agent_data["X"]-measure_point_x)<accepted_dist_delta)].drop_duplicates("AgentID").shape[0]


fr = calc_flow_rate( t_start = t_start,
                t_end = t_end,
                measure_point_x = measure_point_x,
                accepted_delta = accepted_dist_delta)

print("flow rate = ",fr)
print(f"Between {t_start} and {t_end} timestamps, there was {fr} cars at point {measure_point_x} with rounding to {accepted_dist_delta}")


flow rate =  6
Between 800 and 1800 timestamps, there was 6 cars at point 600 with rounding to 10


In [13]:
## Rolling average 

def flow_rate_running_avg(t_start, t_end, measure_point_x, accepted_dist_delta, window_size,running_step=100):
    flow_rates = []
    for t in range(t_start, t_end + 1-window_size,running_step):
        flow_rates.append(calc_flow_rate(t, t + window_size, measure_point_x, accepted_dist_delta)) # there was error so i changed flow_rate to calc_flow_rate
    running_avg = sum(flow_rates) / len(flow_rates)
    print(flow_rates) # check it out, A.
    return running_avg

flow_rate_running_avg(800,5800,600,1000,1000)

[6, 8, 9, 9, 9, 10, 11, 12, 12, 12, 11, 12, 11, 11, 11, 12, 14, 15, 15, 15, 14, 15, 14, 13, 13, 13, 12, 11, 10, 10, 9, 11, 11, 11, 12, 10, 12, 12, 12, 10, 10]


11.463414634146341

## Density
#### Density: Traffic density is the number of cars per unit length of the road at any instant. 

In [14]:
x_start = 500
x_end = 600
time = 2000
def calc_density(x_start,x_end,time):
    return agent_data[(agent_data["Is_alive"]==True)&(agent_data["X"]>x_start)&(agent_data["X"]<x_end)&(agent_data["Step"]==time)].drop_duplicates("AgentID").shape[0]/(x_end-x_start)
d = calc_density(x_start,x_end,time)

# density at time 2000 between  two given nodes (their x_positions)
print(f"The number of cars between pos_x {x_start} and {x_end}, at time {time}, divided by the distance of {x_end-x_start} is equal {d}")

The number of cars between pos_x 500 and 600, at time 2000, divided by the distance of 100 is equal 0.04


In [15]:
# rolling average

def density_running_avg(x_start,x_end,t_start, t_end, running_step=5):
    densities = []
    for t in range(t_start, t_end +1, running_step):
        densities.append(calc_density(x_start,x_end,t))
    running_avg = sum(densities) / len(densities)
    return running_avg

#running average for the same two nodes, using time range, not specific timestamp this time, the result should be simmilar
density_running_avg(500,600,1800,2400)

0.026446280991735505